In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
print(tf.__version__)

2.8.0


# Variáveis

In [2]:
length=100
batch_size=32
learning_rate=0.000001
epocas=5

In [3]:
dados_teste=np.load('dadosTeste.npy')
dadosTreinoValidacao=np.load('dadosTreinoValidacao.npy')

print(np.shape(dadosTreinoValidacao),
np.shape(dados_teste))

dadosTreinoValidacao[3:40]

(100000,) (8937,)


array([9., 7., 5., 2., 9., 3., 7., 7., 4., 8., 2., 9., 5., 3., 3., 2., 6.,
       6., 1., 4., 9., 7., 5., 2., 9., 4., 7., 8., 4., 7., 2., 9., 5., 3.,
       3., 3., 7.])

In [4]:
percentagem_dados=80

x_train = dadosTreinoValidacao[:80000]

x_validation = dadosTreinoValidacao[80000:]

print(np.shape(x_train), np.shape(x_validation))

(80000,) (20000,)


### Dados treino

In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator

def preparar_targets(dados, length,batch_size):
    d=5
    salto=3
    n=d+salto #8
    targets=[]
    for i in range(len(dados)):
        soma = np.sum(dados[d:n])
        d= d+1
        n= n+1
        targets.append(soma)
    del targets[-5:]
    
    targets = np.expand_dims(targets, axis=1)
    
    new = dados[5:]
    inputs = np.expand_dims(new, axis=1)

    time_series_data = tf.keras.preprocessing.sequence.TimeseriesGenerator(inputs, targets, 
                                                                           length= length, batch_size=batch_size)
    
    return time_series_data

time_series_data=preparar_targets(x_train,length,batch_size)
time_series_data[0][0].shape

(32, 100, 1)

### Dados Validação

In [20]:
validation_data = preparar_targets(x_validation,length,batch_size)
validation_data[0][0].shape

(32, 100, 1)

## Rede feedfoward

In [7]:
length = len(validation_data[0][0][0])
print(length)

100


In [8]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape = (length,1)), tf.keras.layers.Dense(128,activation='relu')])

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(1)) 
    
optimizer=tf.keras.optimizers.Adam(0.00001)
loss=tf.keras.losses.MeanSquaredError()
model.compile(loss=loss, optimizer=optimizer, metrics='mae')

#model.fit(time_series_data, epochs = epocas,validation_data =validation_data, shuffle= True, verbose=2)

test_targets=preparar_targets(dados_teste,length,batch_size)
model.evaluate(test_targets)

276/276 [==============================] - 1s 913us/step - loss: 250.3700 - mae: 15.3431


[250.36996459960938, 15.343058586120605]

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 128)               12928     
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 29,569
Trainable params: 29,569
Non-trainable params: 0
_________________________________________________________________


# Tunning Trial

In [10]:
import keras_tuner as kt

In [11]:
def build_model(hp):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape = (length,1)), tf.keras.layers.Dense(128,activation='relu')])
    
    for i in range(hp.Int('layers',2,6)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_'+str(i), 50,268,step=10),
            activation=hp.Choice('activationf_'+str(i),['relu','sigmoid'])))
    
    model.add(tf.keras.layers.Dense(1)) 
    
    optimizer=tf.keras.optimizers.Adam(hp.Choice('act_',[0.00001,0.000001]))
    loss=tf.keras.losses.MeanSquaredError()
    model.compile(loss=loss, optimizer=optimizer, metrics='mae')
    
    return model

In [12]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,executions_per_trial=1)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json


In [13]:
#tuner.search_space_summary()

In [14]:
tuner.search(time_series_data,epochs=15,validation_data=validation_data)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |?                 |layers
130               |?                 |units_0
tanh              |?                 |activationf_0
190               |?                 |units_1
relu              |?                 |activationf_1
1e-06             |?                 |lr_

Epoch 1/15
2497/2497 [==============================] - 4s 2ms/step - loss: 53.9711 - mae: 5.5087 - val_loss: 10.4799 - val_mae: 2.4894
Epoch 2/15
2497/2497 [==============================] - 4s 2ms/step - loss: 5.9194 - mae: 1.9021 - val_loss: 5.4918 - val_mae: 1.8470
Epoch 3/15
2497/2497 [==============================] - 4s 1ms/step - loss: 4.2691 - mae: 1.6386 - val_loss: 5.0870 - val_mae: 1.7832
Epoch 4/15
1002/2497 [===========>..................] - ETA: 1s - loss: 3.9383 - mae: 1.5706

KeyboardInterrupt: 

In [ ]:
tuner.results_summary()